In [1]:
import os
import glob
import re
import pandas as pd
import numpy as np

### Función que ensambla las predicciones de los mejores modelos encontrados por Hard Voting

In [2]:
def generar_prediccion_final_hard_voting_con_reporte(ruta_carpeta, patron_regex, target_x_envios, 
                                                     nombre_archivo_salida, nombre_archivo_reporte, 
                                                     n_modelos_combinacion=3):
    """
    Genera dos archivos:
    1. Archivo de entrega: Solo IDs de clientes (sin header).
    2. Archivo de reporte: ID de cliente y qué modelos lo votaron.

    Argumentos extra:
        nombre_archivo_reporte: Ruta del CSV detallado (Col1: ID, Col2: Modelos).
    """
    
    # 1. Cargar y procesar archivos
    archivos = glob.glob(os.path.join(ruta_carpeta, "*.csv"))
    if not archivos:
        print("Error: No se encontraron archivos .csv.")
        return

    modelos_por_mes = {}
    
    print("Cargando y procesando modelos...")
    for archivo in archivos:
        nombre_archivo = os.path.basename(archivo)
        match = re.search(patron_regex, nombre_archivo)
        if match:
            modelo = match.group(1)
            try:
                df = pd.read_csv(archivo, usecols=['numero_de_cliente', 'probabilidad'])
                if modelo not in modelos_por_mes:
                    modelos_por_mes[modelo] = []
                modelos_por_mes[modelo].append(df)
            except Exception as e:
                print(f"Advertencia: No se pudo leer {nombre_archivo}: {e}")

    # Lista de tuplas: (Nombre_Modelo, DataFrame_Promedio)
    # Necesitamos guardar el nombre para el reporte final
    lista_modelos_data = []
    
    for modelo_nombre, lista_dfs in modelos_por_mes.items():
        # Promedio de semillas intra-modelo
        df_concat = pd.concat(lista_dfs)
        df_avg = df_concat.groupby('numero_de_cliente', as_index=False)['probabilidad'].mean()
        # Ordenar descendente
        df_avg = df_avg.sort_values('probabilidad', ascending=False).reset_index(drop=True)
        
        lista_modelos_data.append((modelo_nombre, df_avg))

    # Validar cantidad de modelos
    if len(lista_modelos_data) < n_modelos_combinacion:
        print(f"Advertencia: Se esperaban {n_modelos_combinacion} modelos, se encontraron {len(lista_modelos_data)}.")
    
    total_clientes = len(lista_modelos_data[0][1])
    
    # ==============================================================================
    # 2. BÚSQUEDA BINARIA (Optimizada para velocidad - Solo IDs)
    # ==============================================================================
    print(f"Buscando el N óptimo para obtener ~{target_x_envios} envíos finales...")
    
    def calcular_cantidad_salida(input_n):
        """Devuelve solo la CANTIDAD de clientes finales para ser rápido"""
        top_clientes = []
        for _, df in lista_modelos_data:
            # Tomamos los IDs top N (usando values para velocidad numpy)
            seleccion = df.iloc[:input_n]['numero_de_cliente'].values
            top_clientes.extend(seleccion)
        
        ids, conteos = np.unique(top_clientes, return_counts=True)
        umbral_votos = (n_modelos_combinacion // 2) + 1
        return np.sum(conteos >= umbral_votos)

    low = target_x_envios 
    high = total_clientes
    best_N = low
    best_diff = float('inf')

    while low <= high:
        mid_N = (low + high) // 2
        cantidad_real = calcular_cantidad_salida(mid_N)
        
        diff = abs(cantidad_real - target_x_envios)
        
        if diff < best_diff:
            best_diff = diff
            best_N = mid_N
        
        if cantidad_real < target_x_envios:
            low = mid_N + 1 
        elif cantidad_real > target_x_envios:
            high = mid_N - 1 
        else:
            best_N = mid_N
            break

    # ==============================================================================
    # 3. GENERACIÓN DETALLADA (Pase final con nombres de modelos)
    # ==============================================================================
    print("-" * 50)
    print(f"OPTIMIZACIÓN FINALIZADA:")
    print(f"Objetivo (X): {target_x_envios} | N Encontrado: {best_N}")
    
    partes_votos = []
    
    # Recolectamos quién votó a quién usando el N óptimo
    for nombre_mod, df in lista_modelos_data:
        # Tomamos el corte
        corte = df.iloc[:best_N].copy()
        # Agregamos columna con el nombre del modelo
        corte['voto_origen'] = nombre_mod 
        # Solo necesitamos ID y quien votó
        partes_votos.append(corte[['numero_de_cliente', 'voto_origen']])
    
    # Unimos todo
    todos_los_votos = pd.concat(partes_votos)
    
    # Agrupamos por cliente y concatenamos los nombres de los modelos
    resumen = todos_los_votos.groupby('numero_de_cliente').agg(
        cantidad_votos=('voto_origen', 'count'),
        detalle_modelos=('voto_origen', lambda x: " + ".join(sorted(x))) # Ej: "LGBM + XGB + CAT"
    ).reset_index()
    
    # Filtramos por mayoría
    umbral_votos = (n_modelos_combinacion // 2) + 1
    df_final = resumen[resumen['cantidad_votos'] >= umbral_votos].copy()
    
    print(f"Envíos reales generados: {len(df_final)}")
    print("-" * 50)

    # ==============================================================================
    # 4. GUARDADO DE ARCHIVOS
    # ==============================================================================

    # A) Archivo de entrega (Solo IDs, sin header)
    try:
        df_final[['numero_de_cliente']].to_csv(nombre_archivo_salida, index=False, header=False)
        print(f"[OK] Archivo de entrega guardado: {nombre_archivo_salida}")
    except Exception as e:
        print(f"[Error] Guardando entrega: {e}")

    # B) Archivo de reporte (ID, Detalle Modelos, Cantidad Votos)
    try:
        # Guardamos con header para que sea legible
        df_final.to_csv(nombre_archivo_reporte, index=False, sep='\t') 
        # Uso sep='\t' (tab) para que sea fácil de pegar en Excel y leer los nombres largos
        print(f"[OK] Archivo de reporte guardado: {nombre_archivo_reporte}")
    except Exception as e:
        print(f"[Error] Guardando reporte: {e}")

### Generar archivo de salida y archivo detalle para auditoría de ensambles

In [4]:
generar_prediccion_final_hard_voting_con_reporte(
    ruta_carpeta="./resultados/predicciones_mejores/", 
    patron_regex=r"predicciones_probabilidad_Compe_03_(.*?)_varias", 
    target_x_envios=11000, 
    nombre_archivo_salida="entrega_final_formato_bot.csv",
    nombre_archivo_reporte="entrega_final_reporte_votos.csv",
    n_modelos_combinacion=5
)

Cargando y procesando modelos...
Buscando el N óptimo para obtener ~11000 envíos finales...
--------------------------------------------------
OPTIMIZACIÓN FINALIZADA:
Objetivo (X): 11000 | N Encontrado: 11303
Envíos reales generados: 11000
--------------------------------------------------
[OK] Archivo de entrega guardado: entrega_final_formato_bot.csv
[OK] Archivo de reporte guardado: entrega_final_reporte_votos.csv


In [5]:
import glob
import re
import pandas as pd
import numpy as np

def generar_prediccion_final_soft_voting(ruta_carpeta, patron_regex, target_x_envios, nombre_archivo_salida):
    """
    Genera el archivo de entrega final usando SOFT VOTING (Promedio de Probabilidades).
    
    1. Promedia las semillas de cada modelo individualmente.
    2. Promedia los resultados de los distintos modelos (Model A + Model B + ...).
    3. Ordena por probabilidad descendente.
    4. Corta exactamente en 'target_x_envios'.
    5. Guarda el CSV solo con los IDs (sin header).
    """
    
    # 1. Cargar y procesar archivos
    archivos = glob.glob(os.path.join(ruta_carpeta, "*.csv"))
    if not archivos:
        print("Error: No se encontraron archivos .csv.")
        return

    modelos_por_mes = {}
    
    print("Cargando y promediando semillas por modelo...")
    for archivo in archivos:
        nombre_archivo = os.path.basename(archivo)
        match = re.search(patron_regex, nombre_archivo)
        if match:
            modelo = match.group(1)
            try:
                df = pd.read_csv(archivo, usecols=['numero_de_cliente', 'probabilidad'])
                if modelo not in modelos_por_mes:
                    modelos_por_mes[modelo] = []
                modelos_por_mes[modelo].append(df)
            except Exception as e:
                print(f"Advertencia: No se pudo leer {nombre_archivo}: {e}")

    # Lista para guardar los promedios de cada modelo
    lista_dfs_promediados = []
    
    for modelo, lista_dfs in modelos_por_mes.items():
        # A) Promedio Intra-modelo (Semillas)
        df_concat = pd.concat(lista_dfs)
        df_avg = df_concat.groupby('numero_de_cliente', as_index=False)['probabilidad'].mean()
        lista_dfs_promediados.append(df_avg)

    if not lista_dfs_promediados:
        print("Error: No se pudieron procesar modelos.")
        return

    print(f"Modelos únicos identificados: {len(lista_dfs_promediados)}")

    # 2. Soft Voting (Promedio Inter-modelos)
    print("Calculando promedio del ensamble...")
    
    # Concatenamos los promedios de cada modelo (A, B, C...)
    df_ensemble_total = pd.concat(lista_dfs_promediados)
    
    # Agrupamos por cliente y calculamos el promedio final
    # Esto asegura que cada MODELO tenga el mismo peso, independientemente de cuántas semillas tenía
    df_final = df_ensemble_total.groupby('numero_de_cliente', as_index=False)['probabilidad'].mean()
    
    # 3. Ordenamiento (Ranking)
    print("Ordenando ranking final...")
    df_final = df_final.sort_values('probabilidad', ascending=False).reset_index(drop=True)
    
    # 4. Corte (Slicing)
    if target_x_envios > len(df_final):
        print(f"Advertencia: Se pidieron {target_x_envios} envíos, pero solo hay {len(df_final)} clientes únicos.")
        target_x_envios = len(df_final)
        
    df_corte = df_final.head(target_x_envios)
    
    # Información útil sobre el corte
    min_prob_entry = df_corte.iloc[-1]['probabilidad']
    print("-" * 50)
    print(f"RESULTADO SOFT VOTING:")
    print(f"Objetivo de envíos: {target_x_envios}")
    print(f"Probabilidad de corte (Score del cliente #{target_x_envios}): {min_prob_entry:.6f}")
    print("-" * 50)

    # 5. Guardado
    try:
        df_corte[['numero_de_cliente']].to_csv(nombre_archivo_salida, index=False, header=False)
        print(f"[OK] Archivo guardado exitosamente: {nombre_archivo_salida}")
    except Exception as e:
        print(f"[Error] Guardando el archivo: {e}")

In [6]:
# --- EJEMPLO DE USO ---
ruta = "./resultados/predicciones_mejores/"
patron = r"predicciones_probabilidad_Compe_03_(.*?)_varias"
salida = "entrega_soft_voting_11000.csv"
generar_prediccion_final_soft_voting(ruta, patron, 11000, salida)

Cargando y promediando semillas por modelo...
Modelos únicos identificados: 5
Calculando promedio del ensamble...
Ordenando ranking final...
--------------------------------------------------
RESULTADO SOFT VOTING:
Objetivo de envíos: 11000
Probabilidad de corte (Score del cliente #11000): 0.117250
--------------------------------------------------
[OK] Archivo guardado exitosamente: entrega_soft_voting_11000.csv
